In [1]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install konlpy


Installing automake (A dependency for mecab-ko)
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:12

In [2]:
import numpy as np
import pandas as pd
from konlpy.tag import *
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



corpus= [
    "나는 나를 사랑한다.",
    "나는 너를 사랑한다.",
    "너는 나를 사랑한다.",
    "너는 너를 사랑한다.",
    "나는 나를 증오한다.",
    "나는 너를 증오한다.",
    "너는 나를 증오한다.",
    "너는 너를 증오한다."
    ]
df=pd.DataFrame({"id": range(len(corpus)),"corpus": corpus,"label":[0,1,1,0,1,0,0,1]
})                 
df

# posToUse=["NNP","NNG","MAG","NP","VV","VV+EF","VV+EP",'XSV+EC']
posToUse=["NNP","NNG","NP","NNB"]
def getTokens(s):
    global posToUse
    return [ i[0] for i in  Mecab().pos(s) if i[1] in posToUse ]

tokenizer = Tokenizer()
sentences=[getTokens(r) for r in corpus]
tokenizer.fit_on_texts (sentences)
# print(tokenizer.texts_to_sequences(sentences))
sentences=[getTokens(r) for r in corpus]
encoded = tokenizer.texts_to_sequences(sentences)
x_train=pad_sequences(encoded,padding='post')
y_train=df["label"] #keras.utils.to_categorical(df["label"],2)


In [22]:
tokenizer.word_index

{'나': 1, '너': 2, '사랑': 3, '증오': 4}

In [3]:
x_train,y_train

(array([[1, 1, 3],
        [1, 2, 3],
        [2, 1, 3],
        [2, 2, 3],
        [1, 1, 4],
        [1, 2, 4],
        [2, 1, 4],
        [2, 2, 4]], dtype=int32), 0    0
 1    1
 2    1
 3    0
 4    1
 5    0
 6    0
 7    1
 Name: label, dtype: int64)

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.optimizers import RMSprop

vocab_size=x_train.shape[0]
embedding_dim = 2


In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_train,y_train)

(8, 3)
(8,)
[[1 1 3]
 [1 2 3]
 [2 1 3]
 [2 2 3]
 [1 1 4]
 [1 2 4]
 [2 1 4]
 [2 2 4]] 0    0
1    1
2    1
3    0
4    1
5    0
6    0
7    1
Name: label, dtype: int64


In [46]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.3))
model.add(Conv1D(256, 3, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 2)           16        
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 2)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 256)         1792      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [47]:
history = model.fit(x_train, y_train,
                    batch_size=2,
                    epochs=400,
                    verbose=1,
                    validation_data=(x_train, y_train))


Epoch 1/400
4/4 [==============================] - 1s 63ms/step - loss: 0.6956 - accuracy: 0.1833 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 2/400
4/4 [==============================] - 0s 15ms/step - loss: 0.6918 - accuracy: 0.9500 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/400
4/4 [==============================] - 0s 13ms/step - loss: 0.6925 - accuracy: 0.5833 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/400
4/4 [==============================] - 0s 13ms/step - loss: 0.6934 - accuracy: 0.4500 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/400
4/4 [==============================] - 0s 16ms/step - loss: 0.6939 - accuracy: 0.6833 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/400
4/4 [==============================] - 0s 14ms/step - loss: 0.6947 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/400
4/4 [==============================] - 0s 13ms/step - loss: 0.6958 - accuracy: 0.2667 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/400
4/4 [===

In [48]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train loss: 0.25860321521759033
Train accuracy: 1.0


In [49]:
model.predict(x_train)

array([[0.09643888],
       [0.8244191 ],
       [0.7388445 ],
       [0.17884803],
       [0.52817696],
       [0.10472092],
       [0.20025846],
       [0.7391807 ]], dtype=float32)

In [50]:
model.predict_classes(x_train)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1]], dtype=int32)

In [51]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(8, 2)


In [36]:
enc=tokenizer.word_index
ws=enc.keys()
ws=[i for i in ws]


In [40]:
import io

# encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# for num, word in enumerate(encoder.subwords):
#   vec = weights[num+1] # skip 0, it's padding.
#   out_m.write(word + "\n")
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
for i in range(3):
  vec = weights[i+1]
  out_m.write(ws[i] + "\n")
  print(i)
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")

out_v.close()
out_m.close()

0
1
2


In [45]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# http://projector.tensorflow.org